<a href="https://colab.research.google.com/github/AOdrio/Datathon/blob/main/otro%20bolaso.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [89]:
import pandas as pd
import numpy as np

import xgboost as xgb
from matplotlib import pyplot as plt
import seaborn as sns 

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_percentage_error as MAPE
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression as LR
from sklearn.neighbors import KNeighborsRegressor as KNN

SEED = 42

In [63]:
url='https://drive.google.com/file/d/1KH1fB83JDAWcMRCHKFf9SmcGH7MQkupE/view?usp=sharing'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]
df_original = pd.read_csv(url)

In [ ]:
df_original.describe()

In [ ]:
ax = df_original.hist(bins=20, figsize=(15, 10))

In [3]:
def train_val_test_split(df, train_percent, validate_percent, seed):
    np.random.seed(seed)
    perm = np.random.permutation(df.index)
    m = len(df.index)
    train_end = int(train_percent * m)
    validate_end = int(validate_percent * m) + train_end
    train = df.iloc[perm[:train_end]]
    validate = df.iloc[perm[train_end:validate_end]]
    test = df.iloc[perm[validate_end:]]
    return train, validate, test

In [4]:
#train_percent = 0.7
#val_percent = 0.15

#df_train, df_val, df_test = train_val_test_split(df_original,train_percent, val_percent, SEED)

In [64]:
y = df_original.pop("ingreso")
X = df_original.copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=SEED)

In [66]:
print('Todo: ',df_original.shape)
print('Train:',X_train.shape)
#print('Val:  ',df_val.shape)
print('Test: ',X_test.shape)

Todo:  (40000, 24)
Train: (32000, 24)
Test:  (8000, 24)


se remplazan los null por la media de la columna

In [67]:
edad_media = X_test['edad'].mean()

In [68]:
def transfromar_DataFrame (df,edad_media):
  df = df.copy()
  df = df.drop(df.columns[0],axis=1)
  #Remplazar nulls de la edad con la media de la columna
  df['edad'] = df['edad'].fillna(edad_media, inplace=False)

  #Transformar categóricas en numéricas
  df['peorcalificacionBCU'] = (df['peorcalificacionBCU'].
                              str.replace('5','6').
                              str.replace('4','5').
                              str.replace('3','4').
                              str.replace('2B','3').
                              str.replace('2A','2').
                              str.replace('1C','1').
                              str.replace('1A','0'))
  df['peorcalificacionBCU'] = pd.to_numeric(df['peorcalificacionBCU'], downcast='integer')
  return df

In [69]:
X_train = transfromar_DataFrame(X_train,edad_media)
X_test = transfromar_DataFrame(X_test,edad_media)
#df_val = transfromar_DataFrame(df_val,edad_media)

In [8]:
#y_train = df_train.pop("ingreso")
#X_train = df_train.copy()
#y_test = df_test.pop("ingreso")
#X_test = df_test.copy()
#y_val = df_val.pop("ingreso")
#X_val = df_val.copy()

In [118]:
prueba = pd.DataFrame(y_train)
prueba['Quantile_rank'] = pd.qcut(prueba['ingreso'], 15, labels = False)
prueba.head()

,ingreso,Quantile_rank
14307,204011.154267,14
17812,23939.929135,0
11020,79194.678640,11
15158,42770.726469,5
24990,60397.719141,8


In [119]:
prueba.groupby(['Quantile_rank']).mean()

,ingreso
Quantile_rank,
0,25014.523489
1,29378.722815
2,33152.684665
3,37016.291258
4,40741.598417
5,44594.816980
6,48886.291787
7,53691.138867
8,59072.863967


In [120]:
prueba['ingreso_medio'] = prueba.groupby('Quantile_rank')['ingreso'].transform('mean')

In [121]:
valores_medios = prueba.groupby(['Quantile_rank']).mean()

In [122]:
MAPE_ingreso_medio = MAPE(prueba['ingreso'],prueba['ingreso_medio'])
print('MAPE linea base: ', round(MAPE_ingreso_medio *100, 2))

MAPE linea base:  4.91


In [123]:
y_train_2=prueba['Quantile_rank']

In [124]:
from sklearn.ensemble import RandomForestClassifier

In [125]:
rf_classifier = RandomForestClassifier(
                      min_samples_leaf=50,
                      n_estimators=1000,
                      bootstrap=True,
                      oob_score=True,
                      n_jobs=-1,
                      random_state=SEED,
                      max_features='auto')

In [126]:
rf_classifier.fit(X_train,y_train_2)
preds_rf = rf_classifier.predict(X_test)

In [127]:
pred_df = pd.DataFrame(preds_rf, columns=['Quantile_rank'])

In [128]:
MAPE_rf = MAPE(y_test,pd.merge(pred_df, valores_medios, on= 'Quantile_rank', how='left' )['ingreso_medio'])
print('MAPE: ', round(MAPE_rf *100, 2))

MAPE linea base:  17.35
